In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import lfilter
from scipy.signal.windows import hamming

from numpy.linalg import lstsq

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

zip_path="/content/drive/MyDrive/L2_DATASET.zip"

import zipfile,os,glob
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# --- unzip ---
extract_path="/content/voices_dataset"
os.makedirs(extract_path,exist_ok=True)
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped to:",extract_path)
print("Files:",os.listdir(extract_path))

print("Extracted folder content:")
for r,ds,fs in os.walk(extract_path):
    print(r,"→",fs)

# --- collect ONLY wavs ---
audio_dir=os.path.join(extract_path,"L2_DATASET")
all_audio=sorted(glob.glob(os.path.join(audio_dir,"*.wav")))
print("Available audio files:")
for f in all_audio: print(os.path.basename(f))
assert all_audio, "No WAV files found. Check /content/voices_dataset/L2_DATASET"

# ---- Helper: plot/save STFT for one file ----
def plot_stft(file_path, n_fft=1024, hop=256, fmax=5000, show=False, save_path=None):
    y, sr = librosa.load(file_path, sr=None, mono=True)
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop, win_length=n_fft,
                     window='hann', center=True)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    fig = plt.figure(figsize=(10,4))
    librosa.display.specshow(S_db, sr=sr, hop_length=hop, x_axis='time', y_axis='hz')
    plt.ylim(0, min(fmax, sr/2))
    plt.colorbar(format="%+2.0f dB", label="Magnitude (dB)")
    plt.title(f"STFT — {os.path.basename(file_path)}")
    plt.tight_layout()

    if save_path:
        fig.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
    elif show:
        plt.show()
    else:
        plt.close(fig)

# ---- 3) Show STFT for the FIRST WAV (and play it) ----
first_file = all_audio[0]                # <-- use the WAV list
y, sr = librosa.load(first_file, sr=None, mono=True)
display(Audio(y, rate=sr))
plot_stft(first_file, n_fft=1024, hop=256, show=True)

# ---- 4) Loop over ALL WAVs and save plots ----
out_dir = os.path.join(extract_path, "stft_plots")
os.makedirs(out_dir, exist_ok=True)

for fp in all_audio:                     # <-- loop over WAVs, not 'files'
    png_name = os.path.splitext(os.path.basename(fp))[0] + "_stft.png"
    save_to = os.path.join(out_dir, png_name)
    plot_stft(fp, n_fft=1024, hop=256, save_path=save_to)

print(f"Saved {len(all_audio)} spectrograms to: {out_dir}")

run_time = "/content/aah.wav"               # <-- use the WAV list
y, sr = librosa.load(run_time, sr=None, mono=True)
display(Audio(y, rate=sr))
plot_stft(run_time, n_fft=1024, hop=256, show=True)